In [1]:
import sys  
sys.path.insert(0, '..')

In [2]:
import argparse

def get_args():
    parser = argparse.ArgumentParser(description=None)
    parser.add_argument('--input_dim', default=2, type=int, help='dimensionality of input tensor')
    parser.add_argument('--hidden_dim', default=200, type=int, help='hidden dimension of mlp')
    parser.add_argument('--learn_rate', default=1e-3, type=float, help='learning rate')
    parser.add_argument('--total_steps', default=2000, type=int, help='number of gradient steps')
    parser.add_argument('--name', default='pend', type=str, help='only one option right now')
    parser.add_argument('--field_type', default='solenoidal', type=str, help='type of vector field to learn')
    parser.set_defaults(feature=True)
    return parser.parse_args()

In [4]:
from hnn.train import train

if __name__ == "__main__":
    args = get_args()
    model, stats = train(args)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/var/folders/0v/9bqjr621655_pg00r8l1nlzh0000gn/T/ipykernel_78525/1246895753.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor( data['x'], requires_grad=True, dtype=torch.float32)
/var/folders/0v/9bqjr621655_pg00r8l1nlzh0000gn/T/ipykernel_78525/1246895753.py:19: UserWarning: To copy construct from a tensor, it is recommended to use

Final train loss 3.0404e-02 +/- 1.6628e-03
Final test loss 3.1661e-02 +/- 1.6327e-03
